# 03. 축약어 추출 (제공 코드 기반 최종본)

In [ ]:
import json
from dataclasses import dataclass
from pathlib import Path

from pydantic import BaseModel, Field
from langchain_naver import ChatClovaX
from langchain.agents import create_agent
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate


In [ ]:
# STEP 1) raw JSON -> all_chunks 생성 (02_parse_law.ipynb 의존 제거)

@dataclass
class ArticleChunk:
    law_name: str
    law_id: str
    article_num: str
    article_title: str
    content: str


def normalize_to_list(v):
    if v is None:
        return []
    if isinstance(v, dict):
        return [v]
    if isinstance(v, list):
        return v
    return []


def parse_law_data(data: dict) -> list[ArticleChunk]:
    law_name = data['법령']['기본정보']['법령명_한글']
    law_id = str(data['법령']['기본정보']['법령ID'])
    out = []

    for article in normalize_to_list(data['법령']['조문'].get('조문단위')):
        if article.get('조문여부') != '조문':
            continue

        parts = []
        header = str(article.get('조문내용', '')).strip()
        if header:
            parts.append(header)

        for para in normalize_to_list(article.get('항')):
            para_text = str(para.get('항내용', '')).strip()
            if para_text:
                parts.append(para_text)
            for ho in normalize_to_list(para.get('호')):
                ho_text = str(ho.get('호내용', '')).strip()
                if ho_text:
                    parts.append(ho_text)
                for mok in normalize_to_list(ho.get('목')):
                    mok_text = str(mok.get('목내용', '')).strip()
                    if mok_text:
                        parts.append(mok_text)

        out.append(
            ArticleChunk(
                law_name=law_name,
                law_id=law_id,
                article_num=str(article.get('조문번호', '')).strip(),
                article_title=str(article.get('조문제목', '')).strip(),
                content='\n'.join(parts),
            )
        )
    return out


# raw 경로 자동 탐색: notebooks/research_mvp/data/processed/raw 우선, 없으면 data/processed/raw
candidates = [
    Path('notebooks/research_mvp/data/processed/raw'),
    Path('data/processed/raw'),
]
raw_dir = next((d for d in candidates if d.exists()), None)
if raw_dir is None:
    raise FileNotFoundError('raw JSON 경로를 찾지 못했습니다. 01_fetch_law.ipynb를 먼저 실행하세요.')

raw_files = sorted(raw_dir.glob('*.json'))
if not raw_files:
    raise FileNotFoundError(f'raw JSON 파일이 없습니다: {raw_dir}')

all_chunks = []
for fp in raw_files:
    payload = json.loads(fp.read_text(encoding='utf-8'))
    all_chunks.extend(parse_law_data(payload))

print('raw_dir:', raw_dir)
print('raw_files:', len(raw_files))
print('all_chunks:', len(all_chunks))
print('sample:', all_chunks[0].law_name, all_chunks[0].article_num, all_chunks[0].article_title)


In [ ]:
# STEP 2) 제공해주신 코드 기반 축약어 추출 체인

class AbbrOutput(BaseModel):
    abbreviations: dict[str, str] = Field(default_factory=dict)

llm = ChatClovaX(
    model="HCX-005",
    temperature=0.1,
    max_tokens=2048,
)

SYSTEM_PROMPT = """당신은 법률 문서 전문을 분석하여, 그 안에서 약어(또는 축약어)의 **원래 의미**를 추출해야 합니다.

#아래의 원칙을 반드시 지키세요:
1. 반드시 문서 내용에 **명시적으로 등장한 약어 또는 축약어만** 추출합니다.
2. 약어가 정의되지 않은 경우, 절대 추측하지 않습니다.
3. 약어가 정의된 문장은 보통 “(이하 ‘~’이라 한다)” 또는 “(이하 ‘~’라 한다)” 형태로 나타납니다.
4. 약어의 원래 의미에는 다음 요소들이 포함될 수 있습니다:
   - 관련 법령, 조항 번호, 시행령/시행규칙
   - “에 따라”, “에 의한”, “이 정하여 고시하는”, “으로 정하는”, “에서 규정한” 등의 조건문
   - 문장 속 수식어, 제약 조건 등
   이러한 조건은 **절대 생략하지 말고 그대로 포함하세요.**
5. 정의 구문에 "다만", "단서", "예외" 등이 이어지는 경우, 해당 조건도 실제 명칭에 반드시 포함합니다.
6. 출력은 반드시 **파싱 가능한 JSON 형식**으로 반환해야 합니다.

#출력 규칙:
- 반드시 유효한 "JSON" 형식으로만 출력하세요.
- 약어가 전혀 없을 경우에는 정확히 빈 JSON 객체만 출력하세요.
- '(이하 "X"이라 한다)/(이하 "X"라 한다)' 패턴이 없는 일반 정의 항목은 제외하세요.

# 문서 내용:
{chunk}
"""

# agent = create_agent(
#     model=llm,
#     tools=[],
#     system_prompt=SYSTEM_PROMPT,
#     # response_format=AbbrOutput,
# )

prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
])
parser = JsonOutputParser()
chain = prompt | llm | parser


def extract_abbr_from_chunk(chunk: str) -> dict[str, str]:
    # 여기서는 예외를 삼키지 않고 상위 루프로 전달
    result = chain.invoke({"chunk": chunk})

    if not isinstance(result, dict):
        return {}

    # 1) 표준 키
    if isinstance(result.get('abbreviations'), dict):
        return {k: v for k, v in result['abbreviations'].items() if isinstance(k, str) and isinstance(v, str)}

    # 2) 한글 키 대응
    if isinstance(result.get('약어'), dict):
        return {k: v for k, v in result['약어'].items() if isinstance(k, str) and isinstance(v, str)}

    # 3) 래핑된 dict 대응
    for v in result.values():
        if isinstance(v, dict):
            return {k: vv for k, vv in v.items() if isinstance(k, str) and isinstance(vv, str)}

    # 4) 평면 dict 대응
    return {k: v for k, v in result.items() if isinstance(k, str) and isinstance(v, str)}


In [ ]:
# STEP 3) chunk별 추출 + law별 집계 + JSON 저장 (resume + retry 지원)
from collections import defaultdict
from pathlib import Path
import json
import time


def chunk_key(c: ArticleChunk) -> str:
    return f"{c.law_id}:{c.article_num}"


def load_existing_chunk_maps(path: Path) -> dict[str, dict[str, str]]:
    if not path.exists():
        return {}
    try:
        obj = json.loads(path.read_text(encoding='utf-8'))
        return obj if isinstance(obj, dict) else {}
    except Exception:
        return {}


def aggregate_by_law(chunks: list[ArticleChunk], chunk_abbr_maps: dict[str, dict[str, str]]):
    idx = {chunk_key(c): c for c in chunks}
    by_law = defaultdict(dict)

    for ckey, amap in chunk_abbr_maps.items():
        c = idx.get(ckey)
        if c is None:
            continue
        for k, v in (amap or {}).items():
            by_law[c.law_name][k] = v

    return dict(by_law)


def save_abbr_outputs(chunk_abbr_maps, law_abbr_maps):
    out_dir = Path('data/processed')
    out_dir.mkdir(parents=True, exist_ok=True)

    p_chunk = out_dir / 'abbr_maps_by_chunk.json'
    p_law = out_dir / 'abbr_maps_by_law.json'

    p_chunk.write_text(json.dumps(chunk_abbr_maps, ensure_ascii=False, indent=2), encoding='utf-8')
    p_law.write_text(json.dumps(law_abbr_maps, ensure_ascii=False, indent=2), encoding='utf-8')

    return p_chunk, p_law


def is_rate_limit_error(e: Exception) -> bool:
    msg = str(e)
    return ('429' in msg) or ('rate exceeded' in msg.lower()) or ('RateLimitError' in msg)


def run_abbr_extraction_for_chunks(
    chunks: list[ArticleChunk],
    limit: int | None = None,
    resume: bool = True,
    retry_empty: bool = True,
    checkpoint_every: int = 5,
    max_retries: int = 3,
    base_sleep: float = 1.0,
    stop_on_error: bool = False,
):
    out_dir = Path('data/processed')
    out_dir.mkdir(parents=True, exist_ok=True)

    chunk_path = out_dir / 'abbr_maps_by_chunk.json'
    failed_path = out_dir / 'abbr_extract_failed.json'

    existing = load_existing_chunk_maps(chunk_path) if resume else {}
    failed = []

    targets = chunks[:limit] if limit else chunks
    total = len(targets)

    print('resume mode:', resume)
    print('retry_empty:', retry_empty)
    print('already done:', len(existing))
    print('targets:', total)

    processed_new = 0
    for i, c in enumerate(targets, 1):
        ckey = chunk_key(c)

        if resume and ckey in existing:
            # 기존 값이 비어있다면 재시도
            if retry_empty and (existing.get(ckey) == {} or existing.get(ckey) is None):
                pass
            else:
                continue

        success = False
        last_err = None
        for attempt in range(max_retries + 1):
            try:
                abbr = extract_abbr_from_chunk(c.content)
                # 성공 시에만 저장 (빈 dict도 "정상 결과"일 수 있으므로 저장)
                existing[ckey] = abbr if isinstance(abbr, dict) else {}
                success = True
                break
            except KeyboardInterrupt:
                # 중단 시 현재까지 저장하고 종료
                law_map = aggregate_by_law(chunks, existing)
                save_abbr_outputs(existing, law_map)
                failed_path.write_text(json.dumps(failed, ensure_ascii=False, indent=2), encoding='utf-8')
                print('[interrupt] checkpoint saved. rerun with resume=True to continue.')
                raise
            except Exception as e:
                last_err = e
                if is_rate_limit_error(e) and attempt < max_retries:
                    wait = min(base_sleep * (2 ** attempt), 30.0)
                    print(f'[warn] 429 on {ckey}, retry {attempt+1}/{max_retries}, sleep={wait:.1f}s')
                    time.sleep(wait)
                    continue
                break

        if not success:
            failed.append({
                'chunk_key': ckey,
                'law_name': c.law_name,
                'article_num': c.article_num,
                'error': str(last_err)[:500] if last_err else 'unknown',
            })
            # 실패는 existing에 기록하지 않음 -> 다음 run에서 다시 시도됨
            print(f'[error] failed: {ckey} -> {last_err}')
            if stop_on_error:
                law_map = aggregate_by_law(chunks, existing)
                save_abbr_outputs(existing, law_map)
                failed_path.write_text(json.dumps(failed, ensure_ascii=False, indent=2), encoding='utf-8')
                print('stop_on_error=True: checkpoint saved and stopped.')
                break
            continue

        processed_new += 1
        if processed_new % checkpoint_every == 0:
            law_map = aggregate_by_law(chunks, existing)
            save_abbr_outputs(existing, law_map)
            failed_path.write_text(json.dumps(failed, ensure_ascii=False, indent=2), encoding='utf-8')
            print(f'checkpoint saved: +{processed_new} new')

        if i % 10 == 0:
            print(f'progress: seen {i}/{total}, new {processed_new}, total saved {len(existing)}, failed {len(failed)}')

    law_abbr_maps = aggregate_by_law(chunks, existing)
    p_chunk, p_law = save_abbr_outputs(existing, law_abbr_maps)

    failed_path.write_text(json.dumps(failed, ensure_ascii=False, indent=2), encoding='utf-8')

    print('saved chunk:', p_chunk)
    print('saved law:', p_law)
    print('failed log:', failed_path)
    print('failed count:', len(failed))
    return existing, law_abbr_maps, failed


In [ ]:
# STEP 4) 실행
# 중간 에러/중단 후 재실행하면 abbr_maps_by_chunk.json 기준으로 이어서 진행됩니다.

chunk_abbr_maps, law_abbr_maps, failed = run_abbr_extraction_for_chunks(
    all_chunks,
    limit=None,
    resume=True,
    retry_empty=False,
    checkpoint_every=5,
    max_retries=3,
    base_sleep=1.0,
    stop_on_error=False,
)

print('chunk_abbr_maps:', len(chunk_abbr_maps))
print('law_abbr_maps:', {k: len(v) for k, v in law_abbr_maps.items()})
print('failed:', len(failed))
